<div style="display: flex; align-items: left;">
    <a href="https://sites.google.com/corp/google.com/genai-solutions/home?authuser=0">
        <img src="https://storage.googleapis.com/miscfilespublic/Linkedin%20Banner%20%E2%80%93%202.png" style="margin-right">
    </a>
</div>

In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.


# **Open Data QnA: Set up BigQuery Source**

---

This notebook shows how to copy a BigQuery public dataset to your GCP project 


This is accomplished through the three following steps:  
> i. Create a BigQuery dataset in your GCP project

> ii. Create a table in the above dataset

> iii. Copy data from the public dataset to the dataset on your project


### **Change your Kernel to the created .venv with poetry from README.md**

Below is the Kernel how it should look like before you proceed

![Kernel](../utilities/imgs/Kernel%20Changed.png)


## 🔗 **1. Connect Your Google Cloud Project**
Time to connect your Google Cloud Project to this notebook. 

In [ ]:
#@markdown Please fill in the value below with your GCP project ID and then run the cell.
PROJECT_ID = input("Enter the project id (same as your Setup Project) to copy source data in bigquery for this solution")

# Quick input validation
assert PROJECT_ID, "⚠️ Please provide your Google Cloud Project ID"

# Configure gcloud.
!gcloud config set project {PROJECT_ID}
print(f'Project has been set to {PROJECT_ID}')

## 🔐 **2. Authenticate to Google Cloud**
Authenticate to Google Cloud as the IAM user logged into this notebook in order to access your Google Cloud Project.

You can do this within Google Colab or using the Application Default Credentials in the Google Cloud CLI.

In [ ]:
# Authentication step

"""Colab Auth""" 
# from google.colab import auth
# auth.authenticate_user()


"""Jupiter Notebook Auth"""
import google.auth
import os

credentials, project_id = google.auth.default()

os.environ['GOOGLE_CLOUD_QUOTA_PROJECT']=PROJECT_ID
os.environ['GOOGLE_CLOUD_PROJECT']=PROJECT_ID

In [ ]:
#Enable all the required APIs for the COPY

!gcloud services enable \
  cloudapis.googleapis.com \
  compute.googleapis.com \
  iam.googleapis.com \
  bigquery.googleapis.com --project {PROJECT_ID}

## ☁️ **Copy a Public Dataset to your GCP Project**

Copy a table from the public dataset to ask questions against. A sample table is chosen below, feel free to choose a different one. 

Note: BigQuery does not allow tables to be copied across regions in certain cases. Therefore, BQ_DST_REGION is set to be BQ_SRC_REGION. Change this parameter to check if copy to your region of interest is allowed.



In [ ]:
# Details of source Dataset
BQ_SRC_PROJECT = "bigquery-public-data"
BQ_SRC_DATASETS = ["imdb", "imdb"]
BQ_SRC_TABLES_LIST = [["title_principals","title_crew", "title_basics","name_basics"], ["reviews", "title_ratings"]] # [] Specify empty list to copy 'all' tables, or a Specific list, eg: ["table1", "table3", "table10"]
BQ_SRC_REGIONS = ["us", "us"]

# Details of destination Dataset
BQ_DST_PROJECT = PROJECT_ID
BQ_DST_DATASETS =[ "imdb_people", "imdb_ratings"] # List of destinaion dataset names
BQ_DST_REGIONS = BQ_SRC_REGIONS # Change if needed

In [ ]:
def createBQDataset(bq_project_id, dataset_name,dataset_region):
    from google.cloud import bigquery
    import google.api_core 

    client=bigquery.Client(project=PROJECT_ID)

    dataset_ref = f"{bq_project_id}.{dataset_name}"
    

    try:
        client.get_dataset(dataset_ref)
        print("Destination Dataset exists")
    except google.api_core.exceptions.NotFound:
        print("Cannot find the dataset hence creating.......")
        dataset=bigquery.Dataset(dataset_ref)
        dataset.location=dataset_region
        client.create_dataset(dataset)
        
    return dataset_ref

def createBQTable(bq_project_id,dataset_name, table_name):
        from google.cloud import bigquery
        import google.api_core 

        client=bigquery.Client(project=PROJECT_ID)

        table_ref = client.dataset(dataset_name, project=bq_project_id).table(table_name)

        try:
            client.get_table(table_ref)
            print(f"Destination Table {table_ref} exists")
            
        except google.api_core.exceptions.NotFound:
            print(f"Creating the table {table_ref}.......")
            table = bigquery.Table(table_ref)
            client.create_table(table)

        return table_ref



In [ ]:
#Create destination table and copy table data
from google.cloud import bigquery

# Initialize BQ client
client=bigquery.Client(project=PROJECT_ID)

for BQ_SRC_DATASET, BQ_SRC_TABLES, BQ_SRC_REGION, BQ_DST_DATASET, BQ_DST_REGION, in zip(BQ_SRC_DATASETS, BQ_SRC_TABLES_LIST, BQ_SRC_REGIONS, BQ_DST_DATASETS, BQ_DST_REGIONS):
    
    # Create Destination Dataset (If the dataset already exists, delete the dataset (and the tables with in) and create an empty dataset)
    dst_dataset_ref=createBQDataset(BQ_DST_PROJECT,BQ_DST_DATASET,BQ_DST_REGION)

    if not BQ_SRC_TABLES:
        #if tables are not explicitly provided, get the list of tables from bigquery
        dataset_id = f'{BQ_SRC_PROJECT}.{BQ_SRC_DATASET}'
        bq_tables_obj = client.list_tables(dataset_id)
        BQ_SRC_TABLES = [table_obj.table_id for table_obj in bq_tables_obj]
    
    for BQ_SRC_TABLE in BQ_SRC_TABLES:

        dst_table_ref=createBQTable(BQ_DST_PROJECT,BQ_DST_DATASET,BQ_SRC_TABLE)
        src_table_ref = client.dataset(BQ_SRC_DATASET, project=BQ_SRC_PROJECT).table(BQ_SRC_TABLE)

        job_config = bigquery.CopyJobConfig(write_disposition="WRITE_TRUNCATE")

        copy_job = client.copy_table(src_table_ref, dst_table_ref, job_config=job_config)
        # Wait for the job to complete and check for errors
        copy_job.result()  

print('Done!')


### If all the above steps are executed suucessfully, the Bigquery Public dataset should be copied to your GCP project